# Data wrapper

In [4]:
import pandas as pd
from selenium import webdriver
from bs4 import BeautifulSoup
from openpyxl import Workbook
from openpyxl import load_workbook
import time

def extrair_detalhes_pet(url):
    # Configuração do Selenium para usar um driver do Chrome
    options = webdriver.ChromeOptions()
    options.add_argument('--headless')  # Para executar o Chrome em modo headless (sem interface gráfica)
    driver = webdriver.Chrome(options=options)
    
    # Carregar a página
    driver.get(url)
    
    # Obter o código HTML da página após a renderização completa
    page_html = driver.page_source
    
    # Fechar o driver do Selenium
    driver.quit()
    
    # Parsear o HTML com BeautifulSoup
    soup = BeautifulSoup(page_html, 'html.parser')
    
    # Extrair detalhes do pet
    nome_pet = soup.find('div', class_='v-col-md-8').find('span').get_text(strip=True)
    
    status_pet_tag = soup.find('div', class_='v-chip__content')
    status_pet = status_pet_tag.get_text(strip=True) if status_pet_tag else 'Não encontrado'
    
    descricao_pet_tag = soup.find('div', class_='v-alert__content')
    descricao_pet = descricao_pet_tag.get_text(strip=True) if descricao_pet_tag else 'Não encontrado'
    
    abrigo_presente, nome_abrigo, endereco_abrigo = verificar_abrigo(page_html)
    
    url_imagem = extrair_url_imagem(url)
    
    return {'Nome do Pet': nome_pet, 'Status do Pet': status_pet, 'Descrição do Pet': descricao_pet, 
            'Abrigo': nome_abrigo if abrigo_presente else 'Não encontrado', 'Endereço do Abrigo': endereco_abrigo if abrigo_presente else 'Não encontrado', 
            'URL': url, 'URL da Imagem': url_imagem}

def extrair_url_imagem(url):
    # Configuração do Selenium para usar um driver do Chrome
    options = webdriver.ChromeOptions()
    options.add_argument('--headless')  # Para executar o Chrome em modo headless (sem interface gráfica)
    driver = webdriver.Chrome(options=options)
    
    # Carregar a página
    driver.get(url)
    
    # Obter o código HTML da página após a renderização completa
    page_html = driver.page_source
    
    # Fechar o driver do Selenium
    driver.quit()
    
    # Parsear o HTML com BeautifulSoup
    soup = BeautifulSoup(page_html, 'html.parser')
    
    # Encontrar a tag img com a classe v-img__img e extrair a URL da imagem
    img_tag = soup.find('img', class_='v-img__img')
    if img_tag:
        return img_tag['src']
    else:
        return None
    
def extrair_url_imagem_timer(url):
    # Configuração do Selenium para usar um driver do Chrome
    options = webdriver.ChromeOptions()
    options.add_argument('--headless')  # Para executar o Chrome em modo headless (sem interface gráfica)
    driver = webdriver.Chrome(options=options)
    
    # Carregar a página
    driver.get(url)
    
    # Esperar por um curto período para permitir o carregamento completo da página
    time.sleep(4)  # Aguarda 1 segundo (você pode ajustar esse valor conforme necessário)
    
    # Obter o código HTML da página após a renderização completa
    page_html = driver.page_source
    
    # Fechar o driver do Selenium
    driver.quit()
    
    # Parsear o HTML com BeautifulSoup
    soup = BeautifulSoup(page_html, 'html.parser')
    
    # Encontrar a tag img com a classe v-img__img e extrair a URL da imagem
    img_tag = soup.find('img', class_='v-img__img')
    if img_tag:
        return img_tag['src']
    else:
        return None

def verificar_abrigo(html):
    soup = BeautifulSoup(html, 'html.parser')
    
    abrigo_info = soup.find('div', class_='v-list-item-title', string=lambda string: string and string.startswith('Abrigo:'))
    if abrigo_info:
        abrigo = abrigo_info.text.strip().replace('Abrigo:', '').strip()
        endereco = abrigo_info.find_next('div', class_='v-list-item-subtitle').text.strip()
        return True, abrigo, endereco
    
    return False, None, None

# Cria ou carrega o arquivo Excel
try:
    wb = load_workbook('databases.xlsx')
except FileNotFoundError:
    wb = Workbook()
    ws = wb.active
    ws.append(['Nome do Pet', 'Status do Pet', 'Descrição do Pet', 'Abrigo', 'Endereço do Abrigo', 'URL', 'URL da Imagem'])
else:
    ws = wb.active

# Lista para armazenar os dados de cada pet
dados_pets = []

# Varrendo os pets 10, 2710
for pet_id in range(1835, 1837):
    url_pet = f'https://petsrs.com.br/pet/{pet_id}'
    try:
        detalhes_pet = extrair_detalhes_pet(url_pet)
        dados_pets.append(detalhes_pet)
        ws.append([detalhes_pet['Nome do Pet'], detalhes_pet['Status do Pet'], detalhes_pet['Descrição do Pet'],
                   detalhes_pet['Abrigo'], detalhes_pet['Endereço do Abrigo'], detalhes_pet['URL'], detalhes_pet['URL da Imagem']])
        print(f"Pet ID {pet_id} salvo com sucesso.")
    except Exception as e:
        print(f"Erro ao salvar pet ID {pet_id}: {str(e)}")

# Salva as alterações no arquivo Excel
wb.save('databases.xlsx')


Pet ID 1835 salvo com sucesso.
Pet ID 1836 salvo com sucesso.
